# Test Service

Intended to test the service.py evaluator.
Runs the service.py and a simple client.



# Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
import PIL
from flatland.utils.rendertools import RenderTool
import imageio
import os

In [4]:
from IPython.display import clear_output
from IPython.core import display
import ipywidgets as ipw
display.display(display.HTML("<style>.container { width:95% !important; }</style>"))

# Experiments

This has been mostly changed to load envs using `importlib_resources`.  It's getting them from the package "envdata.tests`

In [5]:
# ENV FILE PATH
#env_file = "Test_20_Level_0.pkl"
#env_file = "../../evaluation_visualization/round2/or-0827/Test_23/Level_1.pkl"
#env_file = "../../evaluation_visualization/round2/rl-0827/Test_23/Level_1.pkl"

In [6]:
#if os.path.exists("../env_data"):
#    env_file = "../env_data/tests/Test_2_Level_0.pkl"
#else:
#    env_file = "./env_data/tests/Test_2_Level_0.pkl"

In [7]:
#sPack, sResource = "env_data.tests", "Test_2_Level_0.pkl"
sPack, sResource = "env_data.tests", "Test_9_Level_1.pkl"

In [8]:
#env_file = "../../evaluation_visualization/round2/or-0827/Test_23/Level_1.pkl"
#env_file = "../../evaluation_visualization/round2/rl-0827/Test_23/Level_1.pkl"

In [9]:
import pickle

from flatland.envs.rail_env import RailEnv
from flatland.envs.rail_generators import sparse_rail_generator
from flatland.envs.schedule_generators import sparse_schedule_generator
from flatland.envs.malfunction_generators import malfunction_from_file, no_malfunction_generator
from flatland.envs.rail_generators import rail_from_file
from flatland.envs.schedule_generators import schedule_from_file
from flatland.core.env_observation_builder import DummyObservationBuilder

In [10]:
from flatland.envs.persistence import RailEnvPersister
from flatland.evaluators.client import FlatlandRemoteClient
import redis
import subprocess as sp
import shlex
import time

In [11]:
!pwd

/home/jeremy/projects/aicrowd/rl-trains/flatland5/notebooks


### Clear any old redis keys

In [12]:
oRedis = redis.Redis()

In [13]:
lKeys = oRedis.keys("flatland*")
lKeys

[]

In [14]:
for sKey in lKeys:
    print(sKey)
    oRedis.delete(sKey)

## Service python command
### Kill any old service process

In [15]:
!ps -ef | grep -i python | grep -i flatland.evaluators.service | awk '{print $2}' | xargs kill

In [16]:
#sCmd = "python -m flatland.evaluators.service --test_folder ../env_data/tests/service_test --mergeDir ./tmp/merge --actionDir ./tmp/actions --pickle --missingOnly"
sCmd = "python -m flatland.evaluators.service --test_folder ../env_data/tests/service_test --pickle --verbose"
lsCmd = shlex.split(sCmd)

In [17]:
#wOut = ipw.Output()
#wOut

In [18]:
oPipe = sp.Popen(lsCmd)

In [19]:
type(oPipe)

subprocess.Popen

In [20]:
oPipe.poll()

In [21]:
oFRC = FlatlandRemoteClient(test_envs_root="../env_data/tests/service_test/", verbose=True, use_pickle=True)

Request :  {'type': 'FLATLAND_RL.PING', 'payload': {'version': '2.2.1'}, 'response_channel': 'flatland-rl::FLATLAND_RL_SERVICE_ID::response::c26d0b28a6e1a7c59f3e022fa56a8b77', 'timestamp': 1601382710.231949}
Response :  b'\x80\x03}q\x00(X\x04\x00\x00\x00typeq\x01X\x10\x00\x00\x00FLATLAND_RL.PONGq\x02X\x07\x00\x00\x00payloadq\x03}q\x04u.'


In [22]:
env, env_dict = RailEnvPersister.load_new("../env_data/tests/service_test/Test_0/Level_0.pkl") # env_file)
ldActions = env_dict["actions"]

In [23]:
def my_controller(obs, _env):
    dAct = ldActions[_env._elapsed_steps]
    #_action = {}
    #for _idx, _ in enumerate(_env.agents):
    #    _action[_idx] = np.random.randint(0, 5)
    return dAct

In [24]:
oObsB = DummyObservationBuilder()

In [25]:
oObsB.get()

True

In [26]:
if True:
    episode = 0
    obs = True
    while obs:
        obs, info = oFRC.env_create(
            obs_builder_object=oObsB
        )
        if not obs:
            print("null observation!")
            """
            The remote env returns False as the first obs
            when it is done evaluating all the individual episodes
            """
            break
        print("Episode : {}".format(episode))
        episode += 1

        print(oFRC.env.dones['__all__'])

        while True:
            action = my_controller(obs, oFRC.env)
            time_start = time.time()

            try:
                observation, all_rewards, done, info = oFRC.env_step(action)
                time_diff = time.time() - time_start
                print("Step Time : ", time_diff)
                if done['__all__']:
                    print("Current Episode : ", episode)
                    print("Episode Done")
                    print("Reward : ", sum(list(all_rewards.values())))
                    break
            except TimeoutException as err:
                print("Timeout: ", err)
                break

    print("Evaluation Complete...")
    print(oFRC.submit())

Request :  {'type': 'FLATLAND_RL.ENV_CREATE', 'payload': {}, 'response_channel': 'flatland-rl::FLATLAND_RL_SERVICE_ID::response::c5f9c9736b4a07080d62491b55587291', 'timestamp': 1601382711.3886104}
Response :  b'\x80\x03}q\x00(X\x04\x00\x00\x00typeq\x01X\x1f\x00\x00\x00FLATLAND_RL.ENV_CREATE_RESPONSEq\x02X\x07\x00\x00\x00payloadq\x03}q\x04(X\x0b\x00\x00\x00observationq\x05\x88X\r\x00\x00\x00env_file_pathq\x06X\x12\x00\x00\x00Test_0/Level_0.pklq\x07X\x04\x00\x00\x00infoq\x08}q\t(X\x0f\x00\x00\x00action_requiredq\n}q\x0b(K\x00\x88K\x01\x88K\x02\x88K\x03\x88K\x04\x88K\x05\x88K\x06\x88K\x07\x88K\x08\x88K\t\x88uX\x0b\x00\x00\x00malfunctionq\x0c}q\r(K\x00K\x00K\x01K\x00K\x02K\x00K\x03K\x00K\x04K\x00K\x05K\x00K\x06K\x00K\x07K\x00K\x08K\x00K\tK\x00uX\x05\x00\x00\x00speedq\x0e}q\x0f(K\x00G?\xf0\x00\x00\x00\x00\x00\x00K\x01G?\xf0\x00\x00\x00\x00\x00\x00K\x02G?\xf0\x00\x00\x00\x00\x00\x00K\x03G?\xf0\x00\x00\x00\x00\x00\x00K\x04G?\xf0\x00\x00\x00\x00\x00\x00K\x05G?\xf0\x00\x00\x00\x00\x00\x00K\x06G

Step Time :  0.002225637435913086
Request :  {'type': 'FLATLAND_RL.ENV_STEP', 'payload': {'action': {0: 2, 2: 4, 7: 2}, 'inference_time': 9.1552734375e-05}, 'response_channel': 'flatland-rl::FLATLAND_RL_SERVICE_ID::response::c2cd268a6ed0e12a58f25998f8c3554b', 'timestamp': 1601382711.5016613}
Step Time :  0.0016257762908935547
Request :  {'type': 'FLATLAND_RL.ENV_STEP', 'payload': {'action': {0: 3, 2: 4, 7: 2}, 'inference_time': 0.00015044212341308594}, 'response_channel': 'flatland-rl::FLATLAND_RL_SERVICE_ID::response::0b09b4697f8238b2e9cccba01249c2c2', 'timestamp': 1601382711.5034719}
Step Time :  0.002920389175415039
Request :  {'type': 'FLATLAND_RL.ENV_STEP', 'payload': {'action': {0: 2, 2: 3, 7: 1}, 'inference_time': 0.0001926422119140625}, 'response_channel': 'flatland-rl::FLATLAND_RL_SERVICE_ID::response::896b99e503c5c0cab972616cd329e42b', 'timestamp': 1601382711.5065582}
Step Time :  0.0028586387634277344
Request :  {'type': 'FLATLAND_RL.ENV_STEP', 'payload': {'action': {0: 2, 2

Step Time :  0.003414630889892578
Request :  {'type': 'FLATLAND_RL.ENV_STEP', 'payload': {'action': {4: 2, 2: 2, 7: 1}, 'inference_time': 0.0005331039428710938}, 'response_channel': 'flatland-rl::FLATLAND_RL_SERVICE_ID::response::6aee80b0ce19d0fe4c5808f1410f1c69', 'timestamp': 1601382711.5900652}
Step Time :  0.0028533935546875
Request :  {'type': 'FLATLAND_RL.ENV_STEP', 'payload': {'action': {4: 2, 2: 2, 7: 2}, 'inference_time': 8.606910705566406e-05}, 'response_channel': 'flatland-rl::FLATLAND_RL_SERVICE_ID::response::113b0fd2959caf4bd8c3a6718ccb5379', 'timestamp': 1601382711.5929782}
Step Time :  0.0015716552734375
Request :  {'type': 'FLATLAND_RL.ENV_STEP', 'payload': {'action': {4: 2, 2: 2, 7: 3}, 'inference_time': 9.036064147949219e-05}, 'response_channel': 'flatland-rl::FLATLAND_RL_SERVICE_ID::response::86bd1212963b076771c3ec85742ff3de', 'timestamp': 1601382711.5946329}
Step Time :  0.001630544662475586
Request :  {'type': 'FLATLAND_RL.ENV_STEP', 'payload': {'action': {4: 2, 2: 

Step Time :  0.00203704833984375
Request :  {'type': 'FLATLAND_RL.ENV_STEP', 'payload': {'action': {6: 2, 5: 2, 9: 2}, 'inference_time': 0.00027823448181152344}, 'response_channel': 'flatland-rl::FLATLAND_RL_SERVICE_ID::response::e81331e5a930c5d2733a16c056c4b6f9', 'timestamp': 1601382711.7048793}
Step Time :  0.003120899200439453
Request :  {'type': 'FLATLAND_RL.ENV_STEP', 'payload': {'action': {6: 2, 5: 4, 9: 2}, 'inference_time': 0.00012302398681640625}, 'response_channel': 'flatland-rl::FLATLAND_RL_SERVICE_ID::response::105e1608827c50a5033a3e3a424f96b8', 'timestamp': 1601382711.7080626}
Step Time :  0.0017342567443847656
Request :  {'type': 'FLATLAND_RL.ENV_STEP', 'payload': {'action': {6: 2, 9: 3, 5: 2}, 'inference_time': 0.0003230571746826172}, 'response_channel': 'flatland-rl::FLATLAND_RL_SERVICE_ID::response::83b5dd940cc80eea548cb217af73b8e0', 'timestamp': 1601382711.7101629}
Step Time :  0.0022492408752441406
Request :  {'type': 'FLATLAND_RL.ENV_STEP', 'payload': {'action': {6:

Response :  b'\x80\x03}q\x00(X\x04\x00\x00\x00typeq\x01X\x1f\x00\x00\x00FLATLAND_RL.ENV_CREATE_RESPONSEq\x02X\x07\x00\x00\x00payloadq\x03}q\x04(X\x0b\x00\x00\x00observationq\x05\x88X\r\x00\x00\x00env_file_pathq\x06X\x12\x00\x00\x00Test_0/Level_1.pklq\x07X\x04\x00\x00\x00infoq\x08}q\t(X\x0f\x00\x00\x00action_requiredq\n}q\x0b(K\x00\x88K\x01\x88K\x02\x88K\x03\x88K\x04\x88K\x05\x88K\x06\x88K\x07\x88K\x08\x88K\t\x88uX\x0b\x00\x00\x00malfunctionq\x0c}q\r(K\x00K\x00K\x01K\x00K\x02K\x00K\x03K\x00K\x04K\x00K\x05K\x00K\x06K\x00K\x07K\x00K\x08K\x00K\tK\x00uX\x05\x00\x00\x00speedq\x0e}q\x0f(K\x00G?\xf0\x00\x00\x00\x00\x00\x00K\x01G?\xf0\x00\x00\x00\x00\x00\x00K\x02G?\xf0\x00\x00\x00\x00\x00\x00K\x03G?\xf0\x00\x00\x00\x00\x00\x00K\x04G?\xf0\x00\x00\x00\x00\x00\x00K\x05G?\xf0\x00\x00\x00\x00\x00\x00K\x06G?\xf0\x00\x00\x00\x00\x00\x00K\x07G?\xf0\x00\x00\x00\x00\x00\x00K\x08G?\xf0\x00\x00\x00\x00\x00\x00K\tG?\xf0\x00\x00\x00\x00\x00\x00uX\x06\x00\x00\x00statusq\x10}q\x11(K\x00cflatland.envs.agent_uti

Step Time :  0.002927541732788086
Request :  {'type': 'FLATLAND_RL.ENV_STEP', 'payload': {'action': {0: 2, 2: 4, 7: 3}, 'inference_time': 0.0003199577331542969}, 'response_channel': 'flatland-rl::FLATLAND_RL_SERVICE_ID::response::68881c66dc29d6e7cb6ce811c2e3ebb9', 'timestamp': 1601382711.9067602}
Step Time :  0.002467632293701172
Request :  {'type': 'FLATLAND_RL.ENV_STEP', 'payload': {'action': {0: 2, 2: 2, 7: 4}, 'inference_time': 0.00027680397033691406}, 'response_channel': 'flatland-rl::FLATLAND_RL_SERVICE_ID::response::ab16c703c7aa096b02156ac87913d25b', 'timestamp': 1601382711.909553}
Step Time :  0.003297090530395508
Request :  {'type': 'FLATLAND_RL.ENV_STEP', 'payload': {'action': {0: 3, 2: 2, 7: 2}, 'inference_time': 7.557868957519531e-05}, 'response_channel': 'flatland-rl::FLATLAND_RL_SERVICE_ID::response::856a64b23f2622ac246c11ada4c55236', 'timestamp': 1601382711.9128475}
Step Time :  0.0013720989227294922
Request :  {'type': 'FLATLAND_RL.ENV_STEP', 'payload': {'action': {0: 2

Step Time :  0.0021550655364990234
Request :  {'type': 'FLATLAND_RL.ENV_STEP', 'payload': {'action': {4: 2, 2: 2, 5: 3}, 'inference_time': 0.00014090538024902344}, 'response_channel': 'flatland-rl::FLATLAND_RL_SERVICE_ID::response::4993341b2fbefeaa6e341437311ae933', 'timestamp': 1601382712.014935}
Step Time :  0.0016644001007080078
Request :  {'type': 'FLATLAND_RL.ENV_STEP', 'payload': {'action': {4: 1, 2: 2, 5: 2}, 'inference_time': 9.322166442871094e-05}, 'response_channel': 'flatland-rl::FLATLAND_RL_SERVICE_ID::response::823506896e2531103ef3f2314a17a47b', 'timestamp': 1601382712.0166795}
Step Time :  0.0017285346984863281
Request :  {'type': 'FLATLAND_RL.ENV_STEP', 'payload': {'action': {4: 2, 5: 3, 2: 2}, 'inference_time': 9.5367431640625e-05}, 'response_channel': 'flatland-rl::FLATLAND_RL_SERVICE_ID::response::c08e3917426021a02b4aee646daa4543', 'timestamp': 1601382712.0184975}
Step Time :  0.0038983821868896484
Request :  {'type': 'FLATLAND_RL.ENV_STEP', 'payload': {'action': {4: 

Step Time :  0.008268117904663086
Request :  {'type': 'FLATLAND_RL.ENV_STEP', 'payload': {'action': {6: 2, 9: 3, 5: 2}, 'inference_time': 0.000316619873046875}, 'response_channel': 'flatland-rl::FLATLAND_RL_SERVICE_ID::response::1914fa1d409a47c8ce328fe554144818', 'timestamp': 1601382712.1137772}
Step Time :  0.0011856555938720703
Request :  {'type': 'FLATLAND_RL.ENV_STEP', 'payload': {'action': {6: 2, 9: 3, 5: 2}, 'inference_time': 6.985664367675781e-05}, 'response_channel': 'flatland-rl::FLATLAND_RL_SERVICE_ID::response::e02b585e6115f9aecb77a166f918a31a', 'timestamp': 1601382712.1150243}
Step Time :  0.0011496543884277344
Request :  {'type': 'FLATLAND_RL.ENV_STEP', 'payload': {'action': {6: 2, 9: 3, 5: 2}, 'inference_time': 6.914138793945312e-05}, 'response_channel': 'flatland-rl::FLATLAND_RL_SERVICE_ID::response::d0676d7bfe8e80438d92751c081d6a25', 'timestamp': 1601382712.1162426}
Step Time :  0.0013616085052490234
Request :  {'type': 'FLATLAND_RL.ENV_STEP', 'payload': {'action': {6: 

ValueError: unregistered extension code 164

In [ ]:
oFRC.env

## Render episode stuff

In [ ]:
env, env_dict = RailEnvPersister.load_new(sResource, load_from_package=sPack) # env_file)

# the seed has to match that used to record the episode, in order for the malfunctions to match.
env.reset(random_seed=1001)
oRT = RenderTool(env, show_debug=True)
aImg = oRT.render_env(show_rowcols=True, return_image=True, show_inactive_agents=True)
print(env._max_episode_steps)
PIL.Image.fromarray(aImg)


In [ ]:
loAgs = env_dict["agents"]
lCols =  "initial_direction,direction,initial_position,position".split(",")
pd.DataFrame([ [getattr(oAg, sCol) for sCol in lCols] 
              for oAg in loAgs], columns=lCols)

In [ ]:
pd.DataFrame([ [getattr(oAg, sCol) for sCol in lCols] 
              for oAg in env.agents], columns=lCols)

In [ ]:
pd.DataFrame([ vars(oAg) for oAg in env.agents])

In [ ]:
# from persistence.py
def get_agent_state(env):
    list_agents_state = []
    for iAg, oAg in enumerate(env.agents):
        # the int cast is to avoid numpy types which may cause problems with msgpack
        # in env v2, agents may have position None, before starting
        if oAg.position is None:
            pos = (0, 0)
        else:
            pos = (int(oAg.position[0]), int(oAg.position[1]))
        # print("pos:", pos, type(pos[0]))
        list_agents_state.append(
            [*pos, int(oAg.direction), oAg.malfunction_data["malfunction"]])
      
    return list_agents_state

In [ ]:
expert_actions = env_dict['actions']
episode_states = env_dict['episode']

In [ ]:
pd.DataFrame([ vars(oAg) for oAg in env.agents])

In [ ]:
env_renderer = RenderTool(env, gl="PGL", show_debug=True)

n_agents = env.get_num_agents()
x_dim, y_dim = env.width, env.height
max_steps = env_dict['max_episode_steps']

action_dict = {}
frames = []

# log everything in original state
statuses = []
for a in range(n_agents):
    statuses.append(env.agents[a].status)
pilImg = render_env(env_renderer)
frames.append({
    'image': pilImg,
    'statuses': statuses
    })

step = 0
all_done = False
failed_action_check = False
print("Processing episode steps:")
while not all_done and step < len(expert_actions):
    print(step, end=", ")
    """
    for a in range(n_agents):
      if info['action_required'][a]:
          if step < len(expert_actions):
            if a in expert_actions[step]:
              action = expert_actions[step][a]
            else:
              print("Step {}: agent {} needs action but not provided! only got {}".format(step, a, expert_actions[step]))
          else:
              action = 0

      action_dict.update({a: action})
    """
    
    if step < len(expert_actions):
        dAct = expert_actions[step]
    else:
        dAct = {}
    
    next_obs, all_rewards, done, info = env.step(dAct)
  
    if True:
        # Check that agent states match recorded states
        if get_agent_state(env) == episode_states[step]:
            pass
        else:
            print("MISMATCH")
            failed_action_check = True
            #print("env:", get_agent_state(env))
            #print("epi:", episode_states[step])
            llAgSt = get_agent_state(env)
            llEpSt = episode_states[step]
            for iAg, (lAgSt, lEpSt) in enumerate(zip(llAgSt, llEpSt)):
                if lAgSt != lEpSt:
                    print("Ag:", iAg, "Env: ", lAgSt, "Epi:", lEpSt, end = "; ")
            print("------")

  
    # Force agent states from the recorded states
    if False:
        for idx, agent in enumerate(env.agents):
            #print(episode_states[step][idx])
            rcPos = episode_states[step][idx][0:2]
            #print(idx, rcPos)
            if rcPos == [0,0]:
                agent.position = None
            else:
                agent.position = (*rcPos,) # episode_states[step][idx][0], episode_states[step][idx][1]#, episode_states[step][idx][2]

            agent.malfunction_data["malfunction"] = episode_states[step][idx][3]
            agent.direction = int(episode_states[step][idx][2])

            agent.old_direction = int(episode_states[step-1][idx][2])
            agent.old_position = episode_states[step-1][idx][:2]

    statuses = []
    for a in range(n_agents):
        statuses.append(env.agents[a].status)

    #clear_output(wait=True)
    pilImg = render_env(env_renderer)
    frames.append({
        'image': pilImg,
        'statuses': statuses
      })
    #print("Replaying {}/{}".format(step, max_steps))

    if done['__all__']:
        all_done = True
        max_steps = step + 1
        print("done")

    step += 1

In [ ]:
assert failed_action_check == False, "Realised states did not match stored states."

In [ ]:
env.agents[0]

In [ ]:
[ oAg.malfunction_data for oAg in env_dict["agents"] ][:3]

In [ ]:
env_dict["malfunction"]

In [ ]:
env._max_episode_steps

In [ ]:
sfImg = sResource.replace("pkl", "gif")
imageio.mimsave(sfImg, [d["image"] for d in frames], subrectangles=True)

In [ ]:
sfImg

In [ ]:
display.Image(sfImg)

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual, Play
import ipywidgets as widgets
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
from IPython.display import HTML

display.display(HTML('<link rel="stylesheet" href="//stackpath.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css"/>'))

def plot_func(frame_idx):
    frame = frames[int(frame_idx)]
    display.display(frame['image'])
    #print(frame['statuses'])

if True:
    slider = widgets.FloatSlider(value=0, min=0, max=max_steps, step=1)
    interact(plot_func, frame_idx = slider)

    play = Play(
        max=max_steps,
        value=0,
        step=1,
        interval=250
    )

    widgets.link((play, 'value'), (slider, 'value'))
    widgets.VBox([play])

In [ ]:
import numpy as np

In [ ]:
g3Ep = np.array(episode_states)
np.sum(g3Ep[:,:,3] > 0)

In [ ]:
plt.plot(np.sum(g3Ep[:,:,3]>0, axis=1))
plt.title(sResource + "\nmalfunctioning agents by time step")

In [ ]:
g3Ep.shape

In [ ]:
g3Ep2 = np.array(env.cur_episode)
g3Ep2.shape

In [ ]:
plt.step(range(len(g3Ep2)), np.sum(g3Ep2[:,:,4]==1, axis=1), label="Active")
plt.step(range(len(g3Ep2)), np.sum(g3Ep2[:,:,4]==0, axis=1), label="Ready to depart")
plt.title("env: "+ sResource +"\nActive Agents by timestep")
plt.legend()